In [1]:
import re
import requests
from persiantools.jdatetime import JalaliDate
import json
import csv

In [2]:
def get_stock_groups():
    r = requests.get("http://www.tsetmc.com/Loader.aspx?ParTree=111C1213")
    groups = re.findall(r"\d{2}", r.text)
    return groups


In [3]:
def get_stock_ids(group):
    url = "http://www.tsetmc.com/tsev2/data/InstValue.aspx?g={}&t=g&s=0"
    r = requests.get(url.format(group))
    if r.status_code != 200:
        return list()
    ids = set(re.findall(r"\d{15,20}", r.text))
    return list(ids)


In [4]:
def get_stock_detail(stock_id: str, group_id: int) -> "stock":
    url = "http://www.tsetmc.com/Loader.aspx?ParTree=151311&i={}".format(stock_id)
    r = requests.get(url)
    print(url)
    stock = {"stock_id": stock_id}
    stock["group_name"] = re.findall(r"LSecVal='([\D]*)',", r.text)[0]
    stock["instId"] = re.findall(r"InstrumentID='([\w\d]*)',", r.text)[0]
    stock["insCode"] = (
        stock_id if re.findall(r"InsCode='(\d*)',", r.text)[0] == stock_id else 0
    )
    stock["baseVol"] = float(re.findall(r"BaseVol=([\.\d]*),", r.text)[0])
    try:
        stock["name"] = re.findall(r"LVal18AFC='([\D]*)',", r.text)[0]
    except:
        return
    try:
        stock["title"] = re.findall(r"Title='([\D]*)',", r.text)[0]
    except:
        return
    try:
        stock["sectorPe"] = float(re.findall(r"SectorPE='([\.\d]*)',", r.text)[0])
    except:
        stock["sectorPe"] = None
    try:
        stock["shareCount"] = float(re.findall(r"ZTitad=([\.\d]*),", r.text)[0])
    except:
        stock["shareCount"] = None

    try:
        stock["estimatedEps"] = float(
            re.findall(r"EstimatedEPS='([\.\d]*)',", r.text)[0]
        )
    except:
        stock["estimatedEps"] = None
    stock["group_id"] = group_id
    if stock["name"] == "',DEven='',LSecVal='',CgrValCot='',Flow='',InstrumentID='":
        return False
    return stock

In [5]:
def get_stock_insind_history(stock_id):
    history = list()
    url2 = "http://tsetmc.com/tsev2/data/clienttype.aspx?i={}".format(stock_id)
    print(url2)
    t = requests.get(url2)
    if t.text == "":
        print("Blank url")
        return history
    insind_history_text = t.text
    insind_history_list = insind_history_text.split(";")
    for item in insind_history_list:
        item_list = item.split(",")
        if len(item_list) < 3:
            break
        history_item = {}
        history_item["date"] = item_list[0]
        history_item["ind_buy_count"] = item_list[1]
        history_item["ins_buy_count"] = item_list[2]
        history_item["ind_sell_count"] = item_list[3]
        history_item["ins_sell_count"] = item_list[4]
        history_item["ind_buy_volume"] = item_list[5]
        history_item["ins_buy_volume"] = item_list[6]
        history_item["ind_sell_volume"] = item_list[7]
        history_item["ins_sell_volume"] = item_list[8]
        history_item["ind_buy_value"] = item_list[9]
        history_item["ins_buy_value"] = item_list[10]
        history_item["ind_sell_value"] = item_list[11]
        history_item["ins_sell_value"] = item_list[12]
        history.append(history_item)

    return history

In [8]:
def write_csv(data):
    with open('datainsind.csv', 'a') as file:
        file.write('date,'+'stock_id,'+'group_id,'+ 'name,' + 'ind_buy_count,'+ 'ins_buy_count,' + 'ind_sell_count,' + 'ins_sell_count,' + 'ind_buy_volume,' + 'ins_buy_volume,' + 'ind_sell_volume,' + 'ins_sell_volume,' + 'ind_buy_value,' + 'ins_buy_value,' + 'ind_sell_value,' + 'ins_sell_value' + '\n')
        for stock in data:
            for stockItem in stock["history"]:
                file.write(stockItem["date"] + ',' + str(stock["stock_id"]) + ',' + str(stock["group_id"]) + ',' + str(stock["name"].encode("utf-8"))+ ',' + str(stockItem["ind_buy_count"]) + ',' + str(stockItem["ins_buy_count"]) + ',' + str(stockItem["ind_sell_count"]) + ',' + str(stockItem["ins_sell_count"]) + ',' + str(stockItem["ind_buy_volume"]) + ',' + str(stockItem["ins_buy_volume"]) + ',' + str(stockItem["ind_sell_volume"]) + ',' + str(stockItem["ins_sell_volume"]) + ',' + str(stockItem["ind_buy_value"]) + ',' + str(stockItem["ins_buy_value"]) + ',' + str(stockItem["ind_sell_value"]) + ',' + str(stockItem["ins_sell_value"]) + '\n')

In [9]:
def main():
    print("Start getting groups")
    groups = sorted(list(set(get_stock_groups())))
    print("Getting groups done", len(groups))
    all_stock_data = list()
    counter = 0
    for i, group in enumerate(groups):
        stocks = get_stock_ids(group)
        if len(stocks)==0:
            continue
        print("Group parsed", group)
        for stock_id in stocks:
            counter = counter + 1
            print("Parsed stock count", counter)
            stock = get_stock_detail(stock_id,int(group))
            if stock == False or stock == None or type(stock) is not dict:
                continue
            stock["history"] = get_stock_insind_history(stock_id)
                
            all_stock_data.append(stock) 
#     print(all_stock_data)
#     write_json(all_stock_data)
    write_csv(all_stock_data)


In [10]:
main()

Start getting groups
Getting groups done 67
Group parsed 10
Parsed stock count 1
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=8977369674477111
http://tsetmc.com/tsev2/data/clienttype.aspx?i=8977369674477111
Parsed stock count 2
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22941065011246116
http://tsetmc.com/tsev2/data/clienttype.aspx?i=22941065011246116
Parsed stock count 3
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=42690477960659940
http://tsetmc.com/tsev2/data/clienttype.aspx?i=42690477960659940
Parsed stock count 4
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=9260930191570567
http://tsetmc.com/tsev2/data/clienttype.aspx?i=9260930191570567
Blank url
Group parsed 11
Parsed stock count 5
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=28809886765682162
http://tsetmc.com/tsev2/data/clienttype.aspx?i=28809886765682162
Parsed stock count 6
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=45358789705522696
http://tsetmc.com/tsev2/data/clienttype.aspx?i=453587

Group parsed 23
Parsed stock count 54
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=61424048886782950
http://tsetmc.com/tsev2/data/clienttype.aspx?i=61424048886782950
Blank url
Parsed stock count 55
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=30007580613586002
http://tsetmc.com/tsev2/data/clienttype.aspx?i=30007580613586002
Blank url
Parsed stock count 56
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=284478000000000
Parsed stock count 57
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=15082904528024302
http://tsetmc.com/tsev2/data/clienttype.aspx?i=15082904528024302
Blank url
Parsed stock count 58
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=65490886290565185
http://tsetmc.com/tsev2/data/clienttype.aspx?i=65490886290565185
Parsed stock count 59
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=33683240001985963
http://tsetmc.com/tsev2/data/clienttype.aspx?i=33683240001985963
Parsed stock count 60
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=491887295269

Parsed stock count 108
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=46348559193224090
http://tsetmc.com/tsev2/data/clienttype.aspx?i=46348559193224090
Parsed stock count 109
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=19367527798307032
http://tsetmc.com/tsev2/data/clienttype.aspx?i=19367527798307032
Parsed stock count 110
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=28493762200787421
http://tsetmc.com/tsev2/data/clienttype.aspx?i=28493762200787421
Blank url
Parsed stock count 111
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=66772024744156373
http://tsetmc.com/tsev2/data/clienttype.aspx?i=66772024744156373
Parsed stock count 112
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=40808043719554948
http://tsetmc.com/tsev2/data/clienttype.aspx?i=40808043719554948
Parsed stock count 113
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=41713045190742691
http://tsetmc.com/tsev2/data/clienttype.aspx?i=41713045190742691
Parsed stock count 114
http://www.tsetmc.com/Loa

http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=45062188442385800
http://tsetmc.com/tsev2/data/clienttype.aspx?i=45062188442385800
Parsed stock count 163
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=56006915451245411
http://tsetmc.com/tsev2/data/clienttype.aspx?i=56006915451245411
Parsed stock count 164
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=10587006028472176
http://tsetmc.com/tsev2/data/clienttype.aspx?i=10587006028472176
Blank url
Parsed stock count 165
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=62258804563636993
http://tsetmc.com/tsev2/data/clienttype.aspx?i=62258804563636993
Parsed stock count 166
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=54676885047867737
http://tsetmc.com/tsev2/data/clienttype.aspx?i=54676885047867737
Parsed stock count 167
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=36394683912545018
http://tsetmc.com/tsev2/data/clienttype.aspx?i=36394683912545018
Parsed stock count 168
http://www.tsetmc.com/Loader.aspx?ParTree=151311

Parsed stock count 216
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=63915926161403347
http://tsetmc.com/tsev2/data/clienttype.aspx?i=63915926161403347
Parsed stock count 217
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=64842837716888827
http://tsetmc.com/tsev2/data/clienttype.aspx?i=64842837716888827
Parsed stock count 218
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22903901709044823
http://tsetmc.com/tsev2/data/clienttype.aspx?i=22903901709044823
Parsed stock count 219
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22645269567765904
http://tsetmc.com/tsev2/data/clienttype.aspx?i=22645269567765904
Blank url
Parsed stock count 220
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=39436183727126211
http://tsetmc.com/tsev2/data/clienttype.aspx?i=39436183727126211
Parsed stock count 221
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=17834623106317041
http://tsetmc.com/tsev2/data/clienttype.aspx?i=17834623106317041
Parsed stock count 222
http://www.tsetmc.com/Loa

...

http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=38436612777305077
Parsed stock count 788
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=53892173815383358
Parsed stock count 789
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=8817369445940242
Parsed stock count 790
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=61951543013892913
Parsed stock count 791
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=24095835622675205
Parsed stock count 792
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=43638198413087069
Parsed stock count 793
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=55848849021200650
Parsed stock count 794
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=52620240501253486
Parsed stock count 795
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=61062392807810466
Group parsed 60
Parsed stock count 796
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22260326095996531
http://tsetmc.com/tsev2/data/clienttype.aspx?i=22260326095996531
Parsed stock count 797


http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=45502027787448135
http://tsetmc.com/tsev2/data/clienttype.aspx?i=45502027787448135
Blank url
Parsed stock count 845
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=69472361926040823
http://tsetmc.com/tsev2/data/clienttype.aspx?i=69472361926040823
Blank url
Parsed stock count 846
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=66830065858417081
http://tsetmc.com/tsev2/data/clienttype.aspx?i=66830065858417081
Parsed stock count 847
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=9609565131304189
http://tsetmc.com/tsev2/data/clienttype.aspx?i=9609565131304189
Parsed stock count 848
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=6479253536750886
http://tsetmc.com/tsev2/data/clienttype.aspx?i=6479253536750886
Parsed stock count 849
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=43356147887115191
http://tsetmc.com/tsev2/data/clienttype.aspx?i=43356147887115191
Group parsed 66
Parsed stock count 850
http://www.tsetmc.com/Load

http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=64695010770758234
http://tsetmc.com/tsev2/data/clienttype.aspx?i=64695010770758234
Parsed stock count 901
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=44157356183396652
http://tsetmc.com/tsev2/data/clienttype.aspx?i=44157356183396652
Blank url
Group parsed 68
Parsed stock count 902
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=31366347648583654
http://tsetmc.com/tsev2/data/clienttype.aspx?i=31366347648583654
Parsed stock count 903
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=64942549055019553
http://tsetmc.com/tsev2/data/clienttype.aspx?i=64942549055019553
Parsed stock count 904
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=70595828753641750
http://tsetmc.com/tsev2/data/clienttype.aspx?i=70595828753641750
Parsed stock count 905
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=24869832924911721
http://tsetmc.com/tsev2/data/clienttype.aspx?i=24869832924911721
Parsed stock count 906
http://www.tsetmc.com/Loader.asp

Parsed stock count 959
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=32845891587040106
http://tsetmc.com/tsev2/data/clienttype.aspx?i=32845891587040106
Parsed stock count 960
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=30852391633490755
http://tsetmc.com/tsev2/data/clienttype.aspx?i=30852391633490755
Parsed stock count 961
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=64707090254488560
http://tsetmc.com/tsev2/data/clienttype.aspx?i=64707090254488560
Parsed stock count 962
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22229150167002259
http://tsetmc.com/tsev2/data/clienttype.aspx?i=22229150167002259
Blank url
Parsed stock count 963
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22215980602177059
http://tsetmc.com/tsev2/data/clienttype.aspx?i=22215980602177059
Parsed stock count 964
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=50220277245668448
http://tsetmc.com/tsev2/data/clienttype.aspx?i=50220277245668448
Blank url
Parsed stock count 965
http://www.tset

Parsed stock count 1011
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=49320769557672822
http://tsetmc.com/tsev2/data/clienttype.aspx?i=49320769557672822
Blank url
Parsed stock count 1012
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=31907674226378369
http://tsetmc.com/tsev2/data/clienttype.aspx?i=31907674226378369
Blank url
Group parsed 73
Parsed stock count 1013
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=69847139870135237
http://tsetmc.com/tsev2/data/clienttype.aspx?i=69847139870135237
Parsed stock count 1014
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=43362635835198978
http://tsetmc.com/tsev2/data/clienttype.aspx?i=43362635835198978
Parsed stock count 1015
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=14079693677610396
http://tsetmc.com/tsev2/data/clienttype.aspx?i=14079693677610396
Blank url
Parsed stock count 1016
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=63830424809501048
http://tsetmc.com/tsev2/data/clienttype.aspx?i=63830424809501048
Group 

In [6]:
def get_stock_insind_history(stock_id):
    history = list()
    url2 = "http://tsetmc.com/tsev2/data/clienttype.aspx?i={}".format(stock_id)
#     print(url2)
    try:
        t = requests.get(url2)
        if t.text == "":
            print("Blank url")
            return history
        insind_history_text = t.text
        insind_history_list = insind_history_text.split(";")
        for item in insind_history_list:
            item_list = item.split(",")
            if len(item_list) < 3:
                break
            history_item = {}
            history_item["date"] = item_list[0]
            history_item["ind_buy_count"] = item_list[1]
            history_item["ins_buy_count"] = item_list[2]
            history_item["ind_sell_count"] = item_list[3]
            history_item["ins_sell_count"] = item_list[4]
            history_item["ind_buy_volume"] = item_list[5]
            history_item["ins_buy_volume"] = item_list[6]
            history_item["ind_sell_volume"] = item_list[7]
            history_item["ins_sell_volume"] = item_list[8]
            history_item["ind_buy_value"] = item_list[9]
            history_item["ins_buy_value"] = item_list[10]
            history_item["ind_sell_value"] = item_list[11]
            history_item["ins_sell_value"] = item_list[12]
            history.append(history_item)
    except:
        print("************ ERROR:", stock_id)
    return history

def get_lateset_InsInd_data():


    print("Start getting groups")
    groups = sorted(list(set(get_stock_groups())))
    print("Getting groups done", len(groups))
    all_stock_data = list()
    
    counter = 0
    cnt = 0
    for i, group in enumerate(groups):
        cnt += 1
#         if cnt == 7:
#             break
        stocks = get_stock_ids(group)
        if len(stocks)==0:
            continue
        print("Group parsed", group)
        for stock_id in stocks:
            counter = counter + 1
            print("Parsed stock count", counter)
            stock = get_stock_detail(stock_id,int(group))
            if stock == False or stock == None or type(stock) is not dict:
                continue
            stock["history"] = get_stock_insind_history(stock_id)

            all_stock_data.append(stock) 

    L = list()
    STOCK_IDS = list()
    NAMES = list()
    DATES = list()
    IDS = list()
    ind_buy_count = list()
    ins_buy_count = list()
    ind_sell_count = list()
    ins_sell_count = list()
    ind_buy_volume = list()
    ins_buy_volume = list()
    ind_sell_volume = list()
    ins_sell_volume = list()
    ind_buy_value = list()
    ins_buy_value = list()
    ind_sell_value = list()
    ins_sell_value = list()
    for stock in all_stock_data:
        for stockItem in stock["history"]:
    #         print(stockItem['date'])
            IDS.append(stock['stock_id'])
            NAMES.append(stock['name'])
            DATES.append(stockItem['date'])
            ind_buy_count.append(stockItem['ind_buy_count'])
            ins_buy_count.append(stockItem['ins_buy_count'])
            ind_sell_count.append(stockItem['ind_sell_count'])
            ins_sell_count.append(stockItem['ins_sell_count'])
            ind_buy_volume.append(stockItem['ind_buy_volume'])
            ins_buy_volume.append(stockItem['ins_buy_volume'])
            ind_sell_volume.append(stockItem['ind_sell_volume'])
            ins_sell_volume.append(stockItem['ins_sell_volume'])
            ind_buy_value.append(stockItem['ind_buy_value'])
            ins_buy_value.append(stockItem['ins_buy_value'])
            ind_sell_value.append(stockItem['ind_sell_value'])
            ins_sell_value.append(stockItem['ins_sell_value'])

#         L.append(list(zip(IDS, NAMES, DATES, ind_buy_count, ins_buy_count, ind_sell_count, ins_sell_count, ind_buy_volume, ins_buy_volume, ind_sell_volume, ins_sell_volume, ind_buy_value, ins_buy_value, ind_sell_value, ins_sell_value)))
    data = pd.DataFrame(columns=['ID', 'Name', 'Date', 'ind_buy_count', 'ins_buy_count', 'ind_sell_count', 'ins_sell_count', 'ind_buy_volume', 'ins_buy_volume', 'ind_sell_volume', 'ins_sell_volume', 'ind_buy_value', 'ins_buy_value', 'ind_sell_value', 'ins_sell_value'])
    
    data['ID'] = IDS
    data['Name'] = NAMES
    data['Date'] = DATES
    data['ind_buy_count'] = ind_buy_count
    data['ins_buy_count'] = ins_buy_count
    data['ind_sell_count'] = ind_sell_count
    data['ins_sell_count'] = ins_sell_count
    data['ind_buy_volume'] = ind_buy_volume
    data['ins_buy_volume'] = ins_buy_volume
    data['ind_sell_volume'] = ind_sell_volume
    data['ins_sell_volume'] = ins_sell_volume
    data['ind_buy_value'] = ind_buy_value
    data['ins_buy_value'] = ins_buy_value
    data['ind_sell_value'] = ind_sell_value
    data['ins_sell_value'] = ins_sell_value
    
    X = data.iloc[:, 2].apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8])
    data['Date'] = X
    data['Date'] = pd.to_datetime(data['Date'], infer_datetime_format=True)
    return data

In [7]:
indsind = get_lateset_InsInd_data()

Start getting groups
Getting groups done 70
Group parsed 10
Parsed stock count 1
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=9260930191570567
Blank url
Parsed stock count 2
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=42690477960659940
Parsed stock count 3
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=8977369674477111
Parsed stock count 4
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22941065011246116
Group parsed 11
Parsed stock count 5
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=15282093177363578
Parsed stock count 6
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=10497143354080476
Blank url
Parsed stock count 7
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=28809886765682162
Parsed stock count 8
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=120750266957340
Parsed stock count 9
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=45358789705522696
Blank url
Group parsed 13
Parsed stock count 10
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i

Parsed stock count 87
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=33931218652865616
Parsed stock count 88
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=35366681030756042
Parsed stock count 89
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=61424048886782950
Blank url
Parsed stock count 90
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=44013656953678055
Parsed stock count 91
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=14073782708315535
Parsed stock count 92
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=30007580613586002
Blank url
Parsed stock count 93
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=35178706978554988
Parsed stock count 94
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=60247433951600827
Group parsed 25
Parsed stock count 95
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=57722642338781674
Parsed stock count 96
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=7235435095059069
Parsed stock count 97
http://www.tsetmc.com/Loader.aspx

Parsed stock count 175
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=20865316761157979
Parsed stock count 176
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=64485827086284311
Parsed stock count 177
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=38547060135156069
Parsed stock count 178
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=12329519546621752
Parsed stock count 179
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=30765727085936322
Parsed stock count 180
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=9457195379270268
Blank url
Parsed stock count 181
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=63965059137798192
Parsed stock count 182
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=3542690854557886
Parsed stock count 183
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=4159532151694984
Parsed stock count 184
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=43026118635839182
Parsed stock count 185
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i

Parsed stock count 263
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=48102697378515196
Parsed stock count 264
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=71285829584745492
Parsed stock count 265
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=14957056743925737
Parsed stock count 266
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=66127247173352975
Parsed stock count 267
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=45446499784751546
Parsed stock count 268
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=68846812677163892
Parsed stock count 269
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=28033133021443774
Parsed stock count 270
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=17834623106317041
Parsed stock count 271
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=42354736493447489
Parsed stock count 272
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=7457232989848872
Parsed stock count 273
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=5818028

Parsed stock count 350
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=2434703913394836
Parsed stock count 351
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=42387718866026650
Parsed stock count 352
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=6131290133202745
Parsed stock count 353
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=59921975187856916
Parsed stock count 354
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=2254054929817435
Parsed stock count 355
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=44850033148208596
Parsed stock count 356
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=31316784129157162
Parsed stock count 357
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22299894048845903
Parsed stock count 358
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=17059960254855208
Parsed stock count 359
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=62404730109947970
Blank url
Parsed stock count 360
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i

...

http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=15039949673085566
Parsed stock count 702
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=25701602342262768
Blank url
Parsed stock count 703
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=35948133957468680
Parsed stock count 704
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=24212636157410845
Parsed stock count 705
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=43951910415124966
Parsed stock count 706
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=204477865140000
Parsed stock count 707
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=7768327419174840
Parsed stock count 708
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=33541897671561960
Parsed stock count 709
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=59399204564407321
Blank url
Parsed stock count 710
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=69994636304448345
Blank url
Parsed stock count 711
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=19

Group parsed 57
Parsed stock count 790
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=1041425595860000
Parsed stock count 791
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=801655534660000
Parsed stock count 792
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=7920014658832193
Parsed stock count 793
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=118244579237682
Parsed stock count 794
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=154610000000000
Parsed stock count 795
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=46830341954511303
Parsed stock count 796
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=22087269603540841
Parsed stock count 797
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=49776615757150035
Parsed stock count 798
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=7769065543658313
Parsed stock count 799
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=778253364357513
Parsed stock count 800
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=1

http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=39604494485558197
Parsed stock count 881
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=1245925241417929
Parsed stock count 882
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=25095385550006158
Parsed stock count 883
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=21851863906501460
Parsed stock count 884
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=38436612777305077
Parsed stock count 885
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=17217689629682758
Parsed stock count 886
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=3370493881245014
Parsed stock count 887
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=12364611634133528
Parsed stock count 888
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=1831214548277647
Parsed stock count 889
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=56881993417852599
Parsed stock count 890
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=40907790180697910
Parsed stock c

Parsed stock count 969
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=611226000000000
Parsed stock count 970
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=13281937213456378
Parsed stock count 971
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=3722699128879020
Parsed stock count 972
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=16369313804633525
Parsed stock count 973
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=62310858666404704
Blank url
Parsed stock count 974
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=44625249840480397
Parsed stock count 975
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=1625149423498289
Parsed stock count 976
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=8303557992090046
Blank url
Parsed stock count 977
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=30443839313522574
Parsed stock count 978
http://www.tsetmc.com/Loader.aspx?ParTree=151311&i=35424116338766901
Parsed stock count 979
http://www.tsetmc.com/Loader.aspx?ParTree=

ConnectionError: HTTPConnectionPool(host='www.tsetmc.com', port=80): Max retries exceeded with url: /Loader.aspx?ParTree=151311&i=17269972595370241 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D3A44DBB08>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))